### Hyperparameter tuning of Deep learning models in Keras 
#Pretrained model with Keras tuner

In [1]:
from keras import applications
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import optimizers

In [3]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 8.4MB/s 


In [7]:
# load CIFAR-10 dataset
(img_train, y_train), (img_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [8]:
from tensorflow.keras import backend
# Normalize pixel values between 0 and 1
x_train = img_train.astype('float32') / 255.0
x_test = img_test.astype('float32') / 255.0

img_rows, img_cols = 32,32
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

In [9]:
# define the model
def model_builder(hp):
    base_model = applications.VGG16(weights='imagenet', pooling='avg', include_top=False)
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = base_model(inputs)
    for i in range(hp.Int('num_layers', 2, 20)):
        x = Dense(units=hp.Int('units' + str(i), 32, 512, 32), activation='relu')(x)
    x = Dense(10, activation='softmax')(x) 
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [10]:
# Instantiate the tuner and perform hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 5,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

58892288/58889256 [==============================] - 1s 0us/step


In [11]:
#Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.
%time tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 10 Complete [00h 03m 48s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.8266000151634216
Total elapsed time: 00h 24m 56s
INFO:tensorflow:Oracle triggered exit
CPU times: user 15min, sys: 6min 11s, total: 21min 12s
Wall time: 24min 56s


In [12]:
model = tuner.get_best_models()

In [13]:
model[0].summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 480)               246240    
_________________________________________________________________
dense_1 (Dense)              (None, 384)               184704    
_________________________________________________________________
dense_2 (Dense)              (None, 160)               61600     
_________________________________________________________________
dense_3 (Dense)              (None, 192)               30912     
_________________________________________________________________
dense_4 (Dense)              (None, 320)              